### 简洁实现线性回归

In [1]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

print(torch.__version__)

1.4.0


In [3]:
num_inputs = 2
num_features = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_features, num_inputs)), dtype=torch.float32)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float32)

### 读取数据

In [6]:
import torch.utils.data as Data

In [8]:
batch_size = 10
# 将特征数据和标签组合成一个tensor，参数的两个tensor第一维度需要一致
datasets = Data.TensorDataset(features, labels)
# 组合一个数据集和一个采样器，并迭代读取数据
data_iter = Data.DataLoader(datasets, batch_size, shuffle=True)

In [9]:
for X, y in data_iter:
    print(X, y)
    break

tensor([[ 0.2987,  0.2747],
        [-0.8541, -0.7054],
        [-1.4531,  1.6530],
        [-0.1276, -0.0873],
        [ 1.3740, -0.3946],
        [ 0.3369, -1.7670],
        [-1.6364, -0.8261],
        [-0.7245,  0.5980],
        [-0.9078, -0.8322],
        [ 0.9861, -1.2537]]) tensor([ 3.8673,  4.8807, -4.3089,  4.2546,  8.3045, 10.8725,  3.7397,  0.7074,
         5.1964, 10.4267])


### 定义模型

In [17]:
import torch.nn

In [18]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(seld, x):
        y = self.linear(x)
        return y
net = LinearNet(num_inputs)
print(net) # 打印网络结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [19]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可传入其他层 
    )

# 写法二
#net = nn.Sequential()
#net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module.....

# 写法三
#from collections import OrderdDict
#net = nn.Sequential(OrderdDict([
#    ('Linear', nn.Linear(num_inputs, 1))
    #('', ........) 添加其他层
#    ]))
print(net)
print(net[0])

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [20]:
for param in net.parameters():
    print(param)


Parameter containing:
tensor([[0.3864, 0.6723]], requires_grad=True)
Parameter containing:
tensor([-0.6153], requires_grad=True)


### 初始化模型参数

In [21]:
from torch.nn import init

In [22]:
init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0) # 也可直接修改bias的data:net[0].bias.data.dill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

###  定义损失函数

In [23]:
loss = nn.MSELoss() # 这里的MSELoss没有乘1/2

### 定义优化算法 

In [24]:
import torch.optim as optim

In [27]:
optimizer = optim.SGD(net.parameters(), lr=0.3)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.3
    momentum: 0
    nesterov: False
    weight_decay: 0
)


### 训练模型

In [29]:
num_epochs = 3
for epoch in range(1, num_epochs+1):
    for X, y in data_iter:
        output = net(X) # y_hat = Xw+b
        l = loss(output, y.view(-1, 1))# 计算损失
        optimizer.zero_grad() #计算完一个batch要清零一次梯度
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000239
epoch 2, loss: 0.000144
epoch 3, loss: 0.000292


In [32]:
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.9993, -3.4007]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1993], requires_grad=True)
